In [1]:
import pandas as pd
import random

In [2]:
data = pd.read_csv('data/train.csv')
data.head()


,id,time,x,y
0,1,0,800,0
1,1,780,780,0
2,1,1572,792,0
3,1,2392,820,0
4,1,3196,804,0


In [3]:
test = pd.DataFrame(columns=data.columns)
train = pd.DataFrame(columns=data.columns)
PART = 0.1
is_equal = False
ids = data.id.unique().tolist()
while not is_equal:
    test_ids = random.sample(list(ids), int(PART * len(ids)))
    train_ids = set(ids) - set(test_ids)

    test = data.loc[data.id.isin(test_ids)]
    train = data.loc[data.id.isin(train_ids)]

    if 0.99 < (test.x.mean() / train.x.mean()) < 1.01:
        is_equal = True

In [4]:
test.x.mean(), train.x.mean()

(614.654009154094, 618.1030446251923)

In [5]:
test.to_csv('data/split_data/test.csv')
train.to_csv('data/split_data/train.csv')

In [6]:
import pandas as pd

train = pd.read_csv('data/split_data/train.csv', index_col=0)
test = pd.read_csv('data/split_data/test.csv', index_col=0)

In [7]:
PREDICT_PART = 100

from main_features.rr_features import StatFeatures

name_cols = ['tension_index_int',
             'mode_int',
             'std_int',
             'mean_int',
             'var_int',
             'pnn50_int',
             'RMSSD_int',
             'ivr_int',
             'vpr_int',
             'papr_int',
             'tension_index_user',
             'mode_user',
             'std_user',
             'mean_user',
             'var_user',
             'pnn50_user',
             'RMSSD_user',
             'ivr_user',
             'vpr_user',
             'papr_user'] + \
            [f'r-r-{i}' for i in range(PREDICT_PART)] + \
            [f'y{i}' for i in range(PREDICT_PART)]
df_train_stats = pd.DataFrame(columns=name_cols)
df_test_stats = pd.DataFrame(columns=name_cols)
df_train_stats.head()

,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,...,y90,y91,y92,y93,y94,y95,y96,y97,y98,y99


In [8]:
from tqdm import tqdm

max_id = train.id.unique().max()
for id_user in train.id.unique():
    print(f'ID {id_user} из {max_id}')
    stats_for_id = StatFeatures(train.loc[train.id == id_user, 'x'], count_intervals=PREDICT_PART).get_statistic()
    for i in tqdm(range(train.loc[train.id == id_user].shape[0] - PREDICT_PART)):
        if train.iloc[i: i + PREDICT_PART, 2].max() == train.iloc[i: i + PREDICT_PART, 2].min():
            continue
        stats_for_interval = StatFeatures(train.iloc[i: i + PREDICT_PART, 2], count_intervals=PREDICT_PART).get_statistic()
        df_train_stats.loc[i, :] = stats_for_interval + stats_for_id + list(train.iloc[i: i + PREDICT_PART, 2].values) + list(
            train.iloc[i: i + PREDICT_PART, 3].values)

df_train_stats

100%|██████████| 42/42 [00:03<00:00, 11.47it/s]
0it [00:00, ?it/s]
100%|██████████| 27/27 [00:01<00:00, 16.12it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 51/51 [00:03<00:00, 14.93it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 54/54 [00:04<00:00, 13.11it/s]
0it [00:00, ?it/s]
100%|██████████| 63/63 [00:03<00:00, 15.85it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 63/63 [00:03<00:00, 17.73it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 17.29it/s]
0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00, 17.75it/s]
0it [00:00, ?it/s]
100%|██████████| 39/39 [00:02<00:00, 15.90it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 149/149 [00:11<00:00, 12.86it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 35/35 [00:02<00:00, 13.76it/s]
0it [00:00, ?it/s]
100%|██████████| 30/30 [00:02<00:00, 13.22it/s]
0it [00:

ID 1 из 275
ID 3 из 275
ID 4 из 275
ID 5 из 275
ID 6 из 275
ID 7 из 275
ID 10 из 275
ID 12 из 275
ID 13 из 275
ID 14 из 275
ID 16 из 275
ID 17 из 275
ID 19 из 275
ID 20 из 275
ID 21 из 275
ID 22 из 275
ID 24 из 275
ID 27 из 275
ID 29 из 275
ID 30 из 275
ID 31 из 275
ID 32 из 275
ID 33 из 275
ID 34 из 275
ID 35 из 275
ID 36 из 275
ID 38 из 275
ID 39 из 275
ID 40 из 275
ID 41 из 275
ID 43 из 275
ID 44 из 275
ID 45 из 275
ID 46 из 275
ID 47 из 275
ID 48 из 275
ID 49 из 275
ID 51 из 275
ID 52 из 275
ID 53 из 275
ID 54 из 275
ID 55 из 275
ID 58 из 275
ID 59 из 275
ID 60 из 275
ID 61 из 275
ID 62 из 275
ID 63 из 275
ID 64 из 275
ID 65 из 275
ID 66 из 275
ID 67 из 275
ID 68 из 275
ID 69 из 275
ID 70 из 275
ID 71 из 275
ID 72 из 275
ID 73 из 275
ID 74 из 275
ID 75 из 275
ID 76 из 275
ID 77 из 275
ID 78 из 275
ID 79 из 275
ID 80 из 275
ID 82 из 275
ID 84 из 275
ID 85 из 275
ID 86 из 275
ID 87 из 275
ID 88 из 275
ID 89 из 275
ID 90 из 275
ID 91 из 275
ID 92 из 275
ID 93 из 275
ID 96 из 275
ID 97

,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,...,y90,y91,y92,y93,y94,y95,y96,y97,y98,y99
0,4.69316,755.102,218.034,729.56,29.8858,43.4343,295.93,7.08763e-05,8.53302e-07,0.000145676,...,0,0,0,0,0,0,0,0,0,0
1,4.69316,755.102,218.499,730.44,29.9133,44.4444,280.25,7.08763e-05,8.53302e-07,0.000145676,...,0,0,0,0,0,0,0,0,0,0
2,4.69284,755.153,218.699,731,29.9177,44.4444,294.181,7.08763e-05,8.53244e-07,0.000145666,...,0,0,0,0,0,0,0,0,0,0
3,4.69284,755.153,218.973,731.64,29.929,45.4545,294.238,7.08763e-05,8.53244e-07,0.000145666,...,0,0,0,0,0,0,0,0,0,0
4,4.69284,755.153,219.083,731.88,29.9343,44.4444,294.196,7.08763e-05,8.53244e-07,0.000145666,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3556,20.389,619.814,251.301,605.44,41.5072,58.5859,353.603,0.000252747,8.86476e-07,0.000742158,...,1,1,1,1,1,1,1,0,0,0
3557,19.9503,619.672,251.3,605.4,41.5097,57.5758,351.332,0.000247253,8.8668e-07,0.000726191,...,1,1,1,1,1,1,0,0,0,0
3558,20.3925,619.705,248.269,609.4,40.7398,56.5657,341.296,0.000252747,8.86632e-07,0.000742288,...,1,1,1,1,1,0,0,0,0,0
3559,20.3962,619.594,247.366,611.48,40.4537,54.5455,335.178,0.000252747,8.86792e-07,0.000742422,...,1,1,1,1,0,0,0,0,0,0


In [9]:
max_id = test.id.unique().max()
for id_user in test.id.unique():
    print(f'ID {id_user} из {max_id}')
    stats_for_id = StatFeatures(test.loc[test.id == id_user, 'x'], count_intervals=PREDICT_PART).get_statistic()
    for i in tqdm(range(test.loc[test.id == id_user].shape[0] - PREDICT_PART)):
        if test.iloc[i: i + PREDICT_PART, 2].max() == test.iloc[i: i + PREDICT_PART, 2].min():
            continue
        stats_for_interval = StatFeatures(test.iloc[i: i + PREDICT_PART, 2], count_intervals=PREDICT_PART).get_statistic()
        df_test_stats.loc[i, :] = stats_for_interval + stats_for_id + list(test.iloc[i: i + PREDICT_PART, 2].values) + list(
            test.iloc[i: i + PREDICT_PART, 3].values)


100%|██████████| 237/237 [00:14<00:00, 16.43it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 98/98 [00:05<00:00, 17.31it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 96/96 [00:05<00:00, 16.86it/s]
0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00, 14.35it/s]


ID 2 из 273
ID 15 из 273
ID 23 из 273
ID 25 из 273
ID 42 из 273
ID 57 из 273
ID 109 из 273
ID 110 из 273
ID 146 из 273
ID 156 из 273
ID 162 из 273
ID 172 из 273
ID 178 из 273
ID 202 из 273
ID 206 из 273
ID 207 из 273
ID 208 из 273
ID 240 из 273
ID 254 из 273
ID 259 из 273
ID 262 из 273
ID 273 из 273


In [10]:
df_train_stats.loc[(df_train_stats.y0 == 1) |
                   (df_train_stats.y1 == 1) |
                    (df_train_stats.y2 == 1) |
                    (df_train_stats.y3 == 1) |
                    (df_train_stats.y4 == 1)]


,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,...,y90,y91,y92,y93,y94,y95,y96,y97,y98,y99
349,12.6289,786.286,81.9115,767.96,10.6661,7.07071,111.21,0.000198598,1.48575e-06,0.000216206,...,0,0,0,0,0,0,0,0,0,0
350,12.6289,786.286,81.7918,768.44,10.6439,7.07071,111.286,0.000198598,1.48575e-06,0.000216206,...,0,0,0,0,0,0,0,0,0,0
351,12.6325,786.059,81.6484,769,10.6175,8.08081,111.33,0.000198598,1.48618e-06,0.000216269,...,0,0,0,0,0,0,0,0,0,0
352,12.6338,785.98,81.579,769.76,10.598,8.08081,111.54,0.000198598,1.48633e-06,0.00021629,...,0,0,0,0,0,0,0,0,0,0
353,12.6338,785.98,81.5524,770.72,10.5813,8.08081,111.745,0.000198598,1.48633e-06,0.00021629,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3516,17.1272,612.274,226.736,558.88,40.5697,76.7677,338.93,0.000209732,1.37018e-06,0.000408314,...,0,0,0,0,0,0,0,0,0,0
3517,17.1256,612.332,226.785,559.08,40.5639,77.7778,338.94,0.000209732,1.37005e-06,0.000408275,...,0,0,0,0,0,0,0,0,0,0
3518,17.1273,612.272,226.676,558.8,40.5648,75.7576,338.862,0.000209732,1.37019e-06,0.000408315,...,0,0,0,0,0,0,0,0,0,0
3519,17.129,612.213,226.768,559.36,40.5407,75.7576,338.827,0.000209732,1.37032e-06,0.000408354,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_test_stats.to_csv('data/split_data/test_stats.csv')
df_train_stats.to_csv('data/split_data/train_stats.csv')